In [1]:
import torch
import torch.nn.functional as F
import numpy as np
from torch.nn import Parameter

from torch.optim import Adam

from treeQN.treeqn_traj_simplest import TreeQN
import image_world
import random

import pandas as pd

In [2]:
train_data, valid_data = image_world.get_data(size = 20)

<h1> Initialize Model

In [3]:
a = torch.zeros(20,20)
b = torch.zeros(20,20)
b[0][0] = 1
F.mse_loss(a,b) #Target Loss should be at least less than this

tensor(0.0025)

In [4]:
input_shape = train_data[0][0].shape# minimum size #train_data[0][0].shape
num_actions = 4
tree_depth = 4
embedding_dim = 64
gamma = 1 
decode_dropout = 0.5
t1 =False#True is Einsum. False +dx 
model = TreeQN(input_shape=input_shape, num_actions=num_actions, tree_depth=tree_depth, embedding_dim=embedding_dim, gamma=gamma,decode_dropout=decode_dropout,t1=t1)
optimizer = Adam(model.parameters(), lr=1e-4)

Addition Transition


<h1>Optional: Pretrain Autoencoder</h1>
(Doesn't seem necessary, useful for testing if decoding Z is possible though)

In [5]:
#image_world.train_autoencoder(model,optimizer,train_data,valid_data,epochs=100,lambda_reg=0)

In [6]:
# #freeze encoder and decoder
# for param in model.encoder.parameters():
#     param.requires_grad = False
# for param in model.decoder.parameters():
#     param.requires_grad = False

In [7]:
# #test_autoencode ability
# def test_autoencoder(model,valid_data):
#     with torch.no_grad():
#         test_sample = random.choice(valid_data)
#         encoding = model.encoder(test_sample[0])
#         decoding = model.decoder(encoding)
#         print('Original:\n', test_sample[0].numpy()[0][0][4:-4, 4:-4])
#         print('Reconstructed:\n', np.round(decoding.numpy()[0][0][4:-4, 4:-4]))

<h1>Train Full Model

In [8]:
all_gradients = []
model.train()  

for epoch in range(3000):  # epochs
    avg_loss = 0
    avg_decode_loss, avg_first_loss, avg_second_loss, avg_third_loss, avg_fourth_loss = 0, 0, 0, 0, 0
    raw_gradients = []

    for t in random.sample(train_data, len(train_data)):  # sample through all data in random order each epoch
        # Get reconstruction loss to help ground abstract state
        decoded_values, transition_probabilities = model(t[0])
        decode_loss = F.mse_loss(decoded_values[0], t[0], reduction='sum')

        # Flatten transition probabilities to then weigh with loss of each predicted state at each layer
        first = transition_probabilities[0].view(-1,1,1,1)
        second = transition_probabilities[1].view(-1,1,1,1)
        third = transition_probabilities[2].view(-1,1,1,1)
        fourth = transition_probabilities[3].view(-1,1,1,1)

        #Weighted Transitions
        # first_loss = (F.mse_loss(decoded_values[1], t[1], reduction='none') * first).sum()
        # second_loss = (F.mse_loss(decoded_values[2], t[2], reduction='none') * second).sum()
        # third_loss = (F.mse_loss(decoded_values[3], t[3], reduction='none') * third).sum()
        # fourth_loss = (F.mse_loss(decoded_values[4], t[4], reduction='none') * fourth).sum()

        #Greedy Policy (Squeezing to eliminate batch and channel dimensions)
        first_loss = (F.mse_loss(decoded_values[1][first.argmax()].squeeze(0),t[1].squeeze(0).squeeze(0)))
        second_loss = (F.mse_loss(decoded_values[2][second.argmax()].squeeze(0),t[2].squeeze(0).squeeze(0)))
        third_loss = (F.mse_loss(decoded_values[3][third.argmax()].squeeze(0),t[3].squeeze(0).squeeze(0)))
        fourth_loss = (F.mse_loss(decoded_values[4][fourth.argmax()].squeeze(0),t[4].squeeze(0).squeeze(0)))

        total_loss = first_loss + second_loss  + third_loss  + fourth_loss + decode_loss

        # break if total loss is nan
        if torch.isnan(total_loss):
            raise ValueError("NAN LOSS")


        avg_decode_loss += decode_loss.item()
        avg_first_loss += first_loss.item()
        avg_second_loss += second_loss.item()
        avg_third_loss += third_loss.item()
        avg_fourth_loss += fourth_loss.item()
        avg_loss += total_loss.item()


        optimizer.zero_grad()
        total_loss.backward()
        # Monitor gradients before clipping and stepping
        all_gradients.append(image_world.store_gradients(model))
        optimizer.step()

    if epoch % 10 == 0: 
        #print just validation
        print(f"Epoch {epoch + 1}, Validation Loss Weighted: {image_world.validate(model, valid_data,weighted=True)}, | Validation Unweighted Avg: {image_world.validate(model, valid_data,weighted=False)/5}")

    #Individual Lossses
    avg_decode_loss = avg_decode_loss / len(train_data)
    avg_first_loss = avg_first_loss / len(train_data)
    avg_second_loss = avg_second_loss / len(train_data)
    avg_third_loss = avg_third_loss / len(train_data)
    avg_fourth_loss = avg_fourth_loss / len(train_data)
    #Full Loss
    avg_train_loss = avg_loss / len(train_data)
    
    # avg_decode_loss = (avg_decode_loss / len(train_data))/0.0025
    # avg_first_loss = (avg_first_loss / len(train_data))/0.0025
    # avg_second_loss = (avg_second_loss / len(train_data))/0.0025
    # avg_third_loss = (avg_third_loss / len(train_data))/0.0025
    # avg_fourth_loss = (avg_fourth_loss / len(train_data))/0.0025
    # #Full Loss
    # avg_train_loss = (avg_loss / len(train_data))/0.0025    
    # if (avg_decode_loss + avg_first_loss + avg_second_loss + avg_third_loss + avg_fourth_loss) < 2:
    #     break

    print(f"Epoch {epoch + 1}, Total Loss: {avg_train_loss}, DLoss: {avg_decode_loss}, A1: {avg_first_loss}, A2: {avg_second_loss}, A3: {avg_third_loss}, A4: {avg_fourth_loss}")


/home/mike/Desktop/TreeQN/BIB/TreeQN_Approach/Notebooks/image_gridworld/image_world.py:129: UserWarning: Using a target size (torch.Size([1, 1, 20, 20])) that is different to the input size (torch.Size([4, 1, 20, 20])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  first_loss = (F.mse_loss(decoded_values[1], t[1], reduction='none') * first).sum().item()
/home/mike/Desktop/TreeQN/BIB/TreeQN_Approach/Notebooks/image_gridworld/image_world.py:130: UserWarning: Using a target size (torch.Size([1, 1, 20, 20])) that is different to the input size (torch.Size([16, 1, 20, 20])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  second_loss = (F.mse_loss(decoded_values[2], t[2], reduction='none') * second).sum().item()
/home/mike/Desktop/TreeQN/BIB/TreeQN_Approach/Notebooks/image_gridworld/image_world.py:131: UserWarning: Using a target size (torch.Size([1, 1, 20, 20])) that is diffe

Epoch 1, Validation Loss Weighted: 7.439789772033691, | Validation Unweighted Avg: 0.19274857640266418
Epoch 1, Total Loss: 1.2369908419522373, DLoss: 1.2200149381702596, A1: 0.003669295147781006, A2: 0.004197894159535116, A3: 0.004483438099057159, A4: 0.004625277180986648
Epoch 2, Total Loss: 0.7894586377523162, DLoss: 0.7737732619047165, A1: 0.003151914122810757, A2: 0.0038053735889579084, A3: 0.004252355118197474, A4: 0.004475733521394432
Epoch 3, Total Loss: 0.2339936514808373, DLoss: 0.2177554703994908, A1: 0.0035026688657870345, A2: 0.003916942335622893, A3: 0.004283590494147079, A4: 0.004534979539246044
Epoch 4, Total Loss: 0.0473525602539832, DLoss: 0.031303209073799236, A1: 0.0035697479129091583, A2: 0.003853677781510421, A3: 0.004185600163923068, A4: 0.004440325301733206
Epoch 5, Total Loss: 0.024513576967133716, DLoss: 0.00882391597154889, A1: 0.0035227768652310427, A2: 0.003758428745310415, A3: 0.004075952674346892, A4: 0.0043325025948103175
Epoch 6, Total Loss: 0.018834892

KeyboardInterrupt: 

In [9]:
#save model
#torch.save(model.state_dict(), 'model.pth')
#load model
#model.load_state_dict(torch.load('model_1500.pth'))

In [10]:
for name, num in all_gradients[-1]:
    print(name +':', round(num, 2))

transition_fun: 0.0
decoder.fc.weight: 0.1
decoder.fc.bias: 0.01
decoder.deconv1.weight: 0.14
decoder.deconv1.bias: 0.02
decoder.deconv2.weight: 0.15
decoder.deconv2.bias: 0.05
decoder.final_conv.weight: 0.29
decoder.final_conv.bias: 0.04
encoder.cnn_encoder.conv1.weight: 0.02
encoder.cnn_encoder.conv1.bias: 0.03
encoder.cnn_encoder.bn1.weight: 0.0
encoder.cnn_encoder.bn1.bias: 0.0
encoder.cnn_encoder.conv2.weight: 0.07
encoder.cnn_encoder.conv2.bias: 0.06
encoder.cnn_encoder.bn2.weight: 0.01
encoder.cnn_encoder.bn2.bias: 0.03
encoder.cnn_encoder.conv3.weight: 0.07
encoder.cnn_encoder.conv3.bias: 0.03
encoder.cnn_encoder.bn3.weight: 0.01
encoder.cnn_encoder.bn3.bias: 0.01
encoder.cnn_encoder.residual_conv.weight: 0.02
encoder.cnn_encoder.residual_conv.bias: 0.02
encoder.linear.weight: 0.14
encoder.linear.bias: 0.01


<h1>View Predictions

In [11]:
image_world.action_viewer(model,start_state=train_data[0][0],actions = [0,1,2,3],shrink=4)

True Original Start State:
 tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])
Max True Original Start State: (7, 8)
Decoded Next States
Action: 0
Next State:
 tensor([[ 0., -0., -0.,  0.,  0., -0.,  0., -0.,  

In [12]:
image_world.view_greedy_path(model,start_state=train_data[0],shrink=4)

True Path
7 8
8 8
8 9
8 10
9 10
True Original Start State:
 tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])
Max True Original Start State: (7, 8)
Decoded Next States
Action: 3
Next State:
 tensor([[ 0., -0.,

In [13]:
action_df = image_world.get_action_df(model,valid_data)
action_df.var()

Action 1    1.322873
Action 2    0.630054
Action 3    1.450892
Action 4    1.317476
dtype: float64

In [14]:
action_df.head(10)

,Action 1,Action 2,Action 3,Action 4
0,3,1,3,1
1,2,3,1,3
2,3,2,0,3
3,0,1,3,2
4,3,3,3,3
5,1,2,1,3
6,0,2,1,3
7,3,1,3,1
8,2,3,1,3
9,2,3,1,3


In [15]:
unique_rows = action_df.drop_duplicates()
len(action_df), len(unique_rows)

(220, 16)